In [1]:
import os
import getpass

os.environ['OPENAI_API_KEY'] = getpass.getpass()

In [4]:
import sqlite3
import json
from langchain.llms import OpenAI
from langchain_openai import ChatOpenAI

from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory

# 모델
llm = ChatOpenAI(temperature=0)

# 사용자 메모리 저장을 위한 딕셔너리
user_memories = {}

# 약물 정보를 데이터베이스에 저장하는 함수
def store_medication_info(user_id, medication_info):
    try:
        conn = sqlite3.connect('medication_management.db')
        cursor = conn.cursor()
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS medications (
                user_id TEXT,
                medication_info TEXT
            )
        ''')
        cursor.execute(
            "INSERT INTO medications (user_id, medication_info) VALUES (?, ?)",
            (user_id, json.dumps(medication_info))
        )
        conn.commit()
    except sqlite3.Error as e:
        print(f"Database error: {e}")
    finally:
        conn.close()

# 약물 정보 요약을 위한 프롬프트 템플릿
medication_prompt = PromptTemplate(
    input_variables=["medication_info"],
    template="사용자가 복용 중인 약물 정보: {medication_info}. 이 정보를 요약해 주세요."
)

# 약물 정보를 요약하는 함수
def summarize_medication_info(medication_info):
    formatted_prompt = medication_prompt.format(medication_info=medication_info)
    summary = llm.invoke(formatted_prompt)
    # print('debugging:',summary)
    return summary.content

# 요약된 내용을 데이터베이스에 저장하는 함수
def store_summary(user_id, summary):
    try:
        conn = sqlite3.connect('medication_management.db')
        cursor = conn.cursor()
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS summaries (
                user_id TEXT,
                summary TEXT
            )
        ''')
        cursor.execute(
            "INSERT INTO summaries (user_id, summary) VALUES (?, ?)",
            (user_id, summary)
        )
        conn.commit()
    except sqlite3.Error as e:
        print(f"Database error: {e}")
    finally:
        conn.close()

# 챗봇 응답을 위한 프롬프트 템플릿
chatbot_prompt = PromptTemplate(
    input_variables=["question", "summary"],
    template="사용자가 물어본 질문: {question}. 이 질문에 대한 답변을 {summary}참고해서 완성된 문장을 생성해 주세요."
)

# 사용자 질문에 대한 응답을 생성하는 함수
def get_user_question_response(user_id, user_question, summary):
    if user_id not in user_memories:
        user_memories[user_id] = ConversationBufferMemory()
    memory = user_memories[user_id]
    formatted_prompt = chatbot_prompt.format(question=user_question, summary=summary)
    response = llm.invoke(formatted_prompt)
    # Save conversation history
    memory.save_context({"input": user_question}, {"output": response.content})
    # print('debuging:', user_question)
    # print('debuging:',response.content)
    return response.content

# 더미 데이터를 사용함.
if __name__ == '__main__':
    # Dummy user ID and medication info
    user_id = 'user123'
    medication_info = {
        'medications': [
            {
                'name': 'Aspirin',
                'dosage': '100mg',
                'frequency': 'Once daily',
                'purpose': 'Blood thinner'
            },
            {
                'name': 'Metformin',
                'dosage': '500mg',
                'frequency': 'Twice daily',
                'purpose': 'Blood sugar control'
            }
        ],
        'health_conditions': [
            'Type 2 Diabetes',
            'Hypertension'
        ]
    }

    
    store_medication_info(user_id, medication_info)

    
    summary = summarize_medication_info(medication_info)
    print("Medication Summary:")
    print(summary)

    
    store_summary(user_id, summary)

    #질의응답 부분   
    # print('Q&A'+'-'*50)
    while True:
        user_question = input("사용자 질문: ")
        if user_question.lower() in ['종료', '오늘은 여기까지', 'exit', 'quit']:
            break
        response = get_user_question_response(user_id, user_question, summary)
        print()
        print('사용자 질문:')
        print(user_question)
        print()
        print("AI 응답:")
        print(response)
        print()


Medication Summary:
사용자는 혈액을 희게 하는 목적으로 아스피린을 매일 100mg 복용하고 있으며, 혈당 조절을 위해 메트포르민을 하루에 두 번 500mg 복용하고 있습니다. 또한 사용자는 2형 당뇨병과 고혈압 질환을 가지고 있습니다.

사용자 질문:
내가 복용하는 약의 부작용은?

AI 응답:
아스피린과 메트포르민을 복용하면서 혈액을 희게하는 효과와 혈당 조절을 위한 효과를 기대할 수 있지만, 부작용 또한 발생할 수 있습니다. 특히 2형 당뇨병과 고혈압을 가진 경우, 약물간 상호작용이나 부작용이 발생할 수 있으니 주의가 필요합니다. 의사나 약사와 상담하여 적절한 복용 방법과 부작용에 대해 더 자세히 알아보시는 것이 좋습니다.

